In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import os,sys,inspect

current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

import ipywidgets as widgets
from IPython.display import display

In [2]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
os.environ["ENV_FILE"] = '../../.env.dev'
from trade.db import TickerReturn, Ticker
from trade.db import establish_connection
establish_connection({
    "database": "trade_development",
    "user": "user",
    "password": "password",
    "host": "db",
    "port": 5432
}).connect()

../../.env.dev
odict_values(['trade_development', 'trade', 'trade', 'db', '5432', 'development', '.env.dev'])
../../.env.dev
odict_values(['trade_development', 'trade', 'trade', 'db', '5432', 'development', '.env.dev'])


TypeError: establish_connection() takes 0 positional arguments but 1 was given

In [ ]:
import pendulum
start_period = pendulum.naive(2019, 10, 1)

In [ ]:
tickers_list = ["T.TO", "BB", "ATD-B.TO", "REAL.TO", "QTRH.TO", "DRM.TO", "MX.TO", "KEY.TO", "FTT.TO", "BAM-A.TO", "BYD.TO", 
                "L"]
exlist = tickers_list

In [ ]:
from trade.configuration import Configuration
exlist = Configuration().scrape().tickers

In [ ]:
tickers = Ticker.select().where(Ticker.ticker.in_(exlist))
base_query = TickerReturn.select(
    TickerReturn.datetime,
    TickerReturn.close,
).where(
    TickerReturn.interval == '1d',
    TickerReturn.datetime > start_period
).order_by(
    TickerReturn.datetime.asc()
).join(Ticker)

In [ ]:
dataframes = []
for ticker in exlist:
    ticker_query = base_query.where(TickerReturn.ticker == Ticker.get(Ticker.ticker == ticker))
    dataframe = pd.read_sql(ticker_query.sql()[0], database_connection(),
        params=ticker_query.sql()[1],
        index_col='datetime'
    )
    df = pd.DataFrame({ticker: dataframe.close}, index=dataframe.index)
    print(ticker)
    print(df.shape)
    dataframes.append(df)
dataframes = pd.concat(dataframes, axis=1) #.dropna()

In [ ]:
ticker_returns = dataframes.pct_change().dropna()

In [ ]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [ ]:
mu = expected_returns.return_model(dataframes, method="capm_return")
S = risk_models.risk_matrix(dataframes, method="oracle_approximating")

In [ ]:
ef = EfficientFrontier(mu, S)

In [ ]:
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

In [ ]:
cleaned_weights

In [ ]:
ef.portfolio_performance(verbose=True)

In [ ]:
from pypfopt import DiscreteAllocation

da = DiscreteAllocation(cleaned_weights, dataframes.iloc[-1], total_portfolio_value=10000)
alloc, leftover = da.lp_portfolio()
print(f"Leftover: ${leftover:.2f}")
pd.Series(alloc).sort_index()